CNN-Image classify

In [ ]:
import tensorflow as tf

In [ ]:
import numpy as np
import cv2
import os
from tqdm import tqdm
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

In [ ]:
from os import listdir
from os.path import isfile, join


In [ ]:
width = 210
num_classes = 3
trainpath = 'dataset/train/'
testpath = 'dataset/validation/'
trainImg = [trainpath+f for f in listdir(trainpath) if listdir(join(trainpath, f))]
testImg = [testpath+f for f in listdir(testpath) if listdir(join(testpath, f))]

In [ ]:
testImg


In [ ]:
def img2data(path):
  rawImgs = []
  labels = []
  c = 0
  for imagePath in (path):
      for item in tqdm(os.listdir(imagePath)):
          file = os.path.join(imagePath, item)
          #print(file)
          c+=1
          l = imagePath.split('/')[2]
          #l=l.split('/')[1]
          if l == 'Golden':
            labels.append([1,0,0])         
          elif l == 'Labrador':
            labels.append([0,1,0])
          elif l == 'Kuvasz':
            labels.append([0,0,1])
            
          if file[-1] =='g':
            img = cv2.imread(file , cv2.COLOR_BGR2RGB)
            img = cv2.resize(img ,(width,width))
            rawImgs.append(img)
  return rawImgs, labels

In [ ]:
x_train, y_train = img2data(trainImg)
x_test, y_test = img2data(testImg)

In [ ]:
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [ ]:
x_train.shape,y_train.shape,x_test.shape, y_test.shape



---



---



Transfer learning with InceptionV3

In [ ]:
from keras.models import Sequential
from keras.models import Model
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard
from keras import optimizers, losses, activations, models
from keras.layers import Convolution2D, Dense, Input, Flatten, Dropout, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, Concatenate
from keras import applications

input_shape = (width, width, 3)
nclass = 3

base_model = tf.keras.applications.InceptionV3(weights='imagenet', 
                                include_top=False, 
                                input_shape=input_shape)
base_model.trainable = False

add_model = Sequential()
add_model.add(base_model)
add_model.add(GlobalAveragePooling2D())
add_model.add(Dropout(0.5))
# add_model.add(Dense(16))
add_model.add(Flatten())
add_model.add(Dense(128, 
                    activation='relu'))
add_model.add(Dense(nclass, 
                    activation='softmax'))

model = add_model

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='categorical_crossentropy'
                , metrics= ['accuracy'], weighted_metrics=['accuracy'])

batch_size = 32
epochs = 20

In [ ]:
history = model.fit(x_train, y_train ,batch_size=batch_size, epochs=epochs ,validation_data=(x_test, y_test))


In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# "Loss"
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()



---



---


This for try out your image

In [ ]:
testpath = 'dataset/test/'
testImg = [testpath+f for f in listdir(testpath) if listdir(join(testpath, f))]
rimg = []
x=0
y=0
for imagePath in (testImg):
    for item in (os.listdir(imagePath)):
        file = os.path.join(imagePath, item)
        if item.split('.')[0] != "":
          img = cv2.imread(file , cv2.COLOR_BGR2RGB)
          ori = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
          img = cv2.resize(img ,(width,width))
          rimg = np.array(img)
          rimg = rimg.astype('float32')
          rimg /= 255
          rimg = np.reshape(rimg ,(1,width,width,3))
          predict = model.predict(rimg)
          label = ['golden','labrador','kuvasz']
          result = label[np.argmax(predict)]
          #print(predict)
          real = str(item).split("_")[0]
          # print('real:',real)
          # print(str(result))
          #plt.imshow(ori)
          if real != result:
            x+=1
            # print(str(item))
            # print(str(result))
            # plt.imshow(ori)
            #print(real,result,i)
          y+=1
          plt.show()
print("test_accuracy :","%.4f" % ((y-x)/y))


In [ ]:
model.save("dogify_v12_in3_9082.h5")

In [ ]:
from keras.models import load_model
import matplotlib.pyplot as plt
from os import listdir
from os.path import isfile, join
import numpy as np
import cv2

model = load_model('dogify_v13_in3_9533.h5')
width = 299